## Load Pickles of checkpoint charges per byte

In [2]:
from os import getcwd, path, pardir
import pickle

Experiment AES

In [5]:
encrypt_charge = pickle.load(open(path.join(getcwd(), "AES", "aes_encrypt_charge.p"), "rb"))
decrypt_charge = pickle.load(open(path.join(getcwd(), "AES", "aes_decrypt_charge.p"), "rb"))

Experiment AES Flash

In [6]:
aes_encrypt_charge = pickle.load(open(path.join(getcwd(), "AES_Flash", "aes_flash_encrypt_charge.p"), "rb"))
aes_decrypt_charge = pickle.load(open(path.join(getcwd(), "AES_Flash", "aes_flash_decrypt_charge.p"), "rb"))
flash_write_charge = pickle.load(open(path.join(getcwd(), "AES_Flash", "aes_flash_write_charge.p"), "rb"))
flash_read_charge = pickle.load(open(path.join(getcwd(), "AES_Flash", "aes_flash_read_charge.p"), "rb"))

Experiment TrustZone

In [8]:
nsc_store_charge = pickle.load(open(path.join(getcwd(), "TrustZone", "trustzone_store_charge.p"), "rb"))
nsc_load_charge = pickle.load(open(path.join(getcwd(), "TrustZone", "trustzone_load_charge.p"), "rb"))

Experiment TrustZone Flash

In [9]:
flash_nsc_store_charge = pickle.load(open(path.join(getcwd(), "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
flash_nsc_load_charge = pickle.load(open(path.join(getcwd(), "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))

In [10]:
application_names = ["Kalman", "AR", "Dijkstra", "Cuckoo", "Sorting"]
application_charges = [1, 2, 3, 4, 5]

In [ ]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(10, 6))